In [1]:
import numpy as np
import wandb 
from layer import layer
from activation import activation
from keras.datasets import fashion_mnist
from loss import loss
from tqdm.auto import tqdm
from optimizer import optimzer

c:\Users\mfaza\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
wandb.login()


  

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [11]:
wandb.init(project='deep learing assignment1 ',name='Mohammed Fazalullah')
wandb.config.update({'learing_rate':0.001,"epochs":10})


In [8]:

(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()
train_images, test_images = train_images/255.0, test_images/255.0
train_images, test_images = train_images.reshape(-1, 784), test_images.reshape(-1, 784)

size=len(train_images)
def one_hot_encode(labels,classes):
    one_hot=np.zeros((size,classes))
    one_hot[np.arange(labels.shape[0]),labels]=1
    return one_hot


y_true=one_hot_encode(train_labels,10)
y_test = one_hot_encode(test_labels, 10)

In [9]:
n=3 #number of layers
layer1=layer(784,16)
layer2=layer(16,16)
layer3=layer(16,10)
activ=activation()
cost=loss()
learning_rate=0.001


In [13]:
epochs = 10
batch_size = 64

num_samples = train_images.shape[0]
num_batches = (num_samples )// batch_size
def train():
    for epoch in tqdm(range(epochs)):
    # indices = np.arange(num_samples)
    # np.random.shuffle(indices)
    # X_train = train_images[indices]
    # print(X_train[0][0])
    # y_train = y_true[indices]

        train_loss, train_acc = [], []
        for batch in range(num_batches):
    
            start = batch * batch_size
            end = start + batch_size

            X_batch = train_images[start:end]
    
            y_batch = one_hot_encode(train_labels[start:end],10)

        
            for i in range(batch_size):
                X_batch[i]=X_batch[i].reshape(1,784)
                z1 = layer1.forward(X_batch[i])
                a1=activ.sgm(z1)
                z2=layer2.forward(a1)
                a2=activ.sgm(z2)
                z3=layer3.forward(a2)
                a3=activ.sftmx(z3)
                a3=np.clip(a3,1e-7,1-1e-7)

                train_probs = a3
       
        
       
                train_loss.append(cost.cross_entropy(train_probs, y_batch[i]))
                train_acc.append(np.mean( np.argmax(train_probs) == np.argmax(y_batch[i])))

                dL_dz3 = train_probs - y_batch[i]
                dL_da2=np.dot(dL_dz3, layer3.weights.T)
                dL_dz2=dL_da2*(z2>0)
                dL_da1=np.dot(dL_dz2, layer2.weights.T)
                dL_dz1=dL_da1*(z1>0)
            

                dw3=np.dot(a2.T,dL_dz3)
                dw2=np.dot(a1.T,dL_dz2)
                dw1=np.dot(X_batch[i].reshape(1,784).T, dL_dz1)

                db3=np.sum(dL_dz3,axis=0,keepdims=True)
                db2=np.sum(dL_dz2,axis=0,keepdims=True)
                db1=np.sum(dL_dz1,axis=0,keepdims=True)
           

            # layer3_update=optimzer(layer3.weights,layer3.biases,0.001,dw3,db3).adagrad()
            # layer2_update=optimzer(layer2.weights,layer2.biases,0.001,dw2,db2).adagrad()
            # layer1_update=optimzer(layer1.weights,layer1.biases,0.001,dw1,db1).adagrad()

                layer3.weights-=(learning_rate*dw3)
                layer2.weights-=(learning_rate*dw2)
                layer1.weights-=(learning_rate*dw1)
                layer3.biases-=(learning_rate*db3)
                layer2.biases-=(learning_rate*db2)
                layer1.biases-=(learning_rate*db1)
            
           
                l=cost.cross_entropy(a3,y_true[i])
        
        train_acc = np.mean(train_acc)
        train_loss = np.mean(train_loss)
        print(f"Epoch {epoch+1}/{epochs} - Loss: {train_loss:.4f}, Accuracy: {train_acc:.4f}")

        print(train_loss, train_acc)


sweep_config = {
    "method": "bayes",  # Bayesian optimization for efficiency
    "metric": {"name": "test_accuracy", "goal": "maximize"},
    "parameters": {
        "epochs": {"values": [5, 10]},
        "hidden_layers": {"values": [3, 4, 5]},
        "hidden_size": {"values": [32, 64, 128]},
        "weight_decay": {"values": [0, 0.0005, 0.5]},
        "learning_rate": {"values": [1e-3, 1e-4]},
        "optimizer": {"values": ["sgd", "momentum", "nesterov", "rmsprop", "adam", "nadam"]},
        "batch_size": {"values": [16, 32, 64]},
        "weight_init": {"values": ["random"]},
        "activation": {"values": ["sigmoid", "tanh", "ReLU"]}
    }
}

sweep_id = wandb.sweep(sweep_config, project="deep learing assignment1 ")
wandb.agent(sweep_id, function=train, count=20)




wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: 7vfmomu0
Sweep URL: https://wandb.ai/mfazal735-iit-madras-foundation/deep%20learing%20assignment1%20/sweeps/7vfmomu0


wandb: Agent Starting Run: e5qyht29 with config:
wandb: 	activation: ReLU
wandb: 	batch_size: 64
wandb: 	epochs: 5
wandb: 	hidden_layers: 4
wandb: 	hidden_size: 128
wandb: 	learning_rate: 0.0001
wandb: 	optimizer: nesterov
wandb: 	weight_decay: 0.0005
wandb: 	weight_init: random
  0%|          | 0/10 [00:00<?, ?it/s]c:\Users\mfaza\OneDrive\Desktop\DL\DL_assignment\activation.py:17: RuntimeWarning: overflow encountered in exp
  return (1/(1+np.exp(-z)))
 10%|█         | 1/10 [00:13<02:02, 13.58s/it]

Epoch 1/10 - Loss: 1.3254, Accuracy: 0.4389
1.3254125427810663 0.4388840715048026


 20%|██        | 2/10 [00:33<02:18, 17.32s/it]

...

100%|██████████| 10/10 [19:16<00:00, 115.69s/it]

Epoch 10/10 - Loss: 1.1765, Accuracy: 0.5201
1.1764685258015153 0.5200940501600854



Exception in thread Thread-122 (_run_job):
Traceback (most recent call last):
  File "c:\Users\mfaza\AppData\Local\Programs\Python\Python311\Lib\site-packages\wandb\agents\pyagent.py", line 307, in _run_job
    wandb.finish()
  File "c:\Users\mfaza\AppData\Local\Programs\Python\Python311\Lib\site-packages\wandb\sdk\wandb_run.py", line 4130, in finish
    wandb.run.finish(exit_code=exit_code, quiet=quiet)
  File "c:\Users\mfaza\AppData\Local\Programs\Python\Python311\Lib\site-packages\wandb\sdk\wandb_run.py", line 449, in wrapper
    return func(self, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\mfaza\AppData\Local\Programs\Python\Python311\Lib\site-packages\wandb\sdk\wandb_run.py", line 391, in wrapper
    return func(self, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\mfaza\AppData\Local\Programs\Python\Python311\Lib\site-packages\wandb\sdk\wandb_run.py", line 2106, in finish
    return self._finish(exit_code)
           ^^^^^^^^

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000002870D796D10>> (for post_run_cell):


ConnectionAbortedError: [WinError 10053] An established connection was aborted by the software in your host machine